<a href="https://colab.research.google.com/github/alier-une/CorpusFromPDF/blob/main/%D0%92%D0%B5%D1%81%D1%82%D0%BD%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задачи
Необходимо создать корпус аннотаций, взятых из статьей по лингвистике, размещенных на сайте Вестник ЮУрГУ.


## Установка необходимых библиотек

In [3]:
# Устанавливаем библиотеку отправки всех видов HTTP-запросов
!pip install requests --quiet

In [4]:
# Устанавливаем библиотеку для парсинга HTML и XML документов
!pip install beautifulsoup4 --quiet

In [5]:
# Устанавливаем библиотеку для считывания файла PDF по пути репозитория
!pip install PyPDF2 --quiet
# Устанавливаем библиотеку для выполнения анализа структуры и извлечения текста и формата из PDF.
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.4 MB/s eta 0:00:00


## Импортирование зависимостей

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
# Для считывания PDF
import PyPDF2

# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

## Создание функции для извлечения текста из пдф-файла

In [8]:
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()

    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []

    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))

    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

## Парсер сайта

In [9]:
start_url = 'https://vestnik.susu.ru/linguistics/issue/archive'

# Получаем данные со стартовой страницы
start_page = requests.get(start_url)
# Изучаем данные со стартовой страницы
soup = BeautifulSoup(start_page.text, 'html.parser')

      #links = soup.find_all('a') находим все гиперссылки

# Выбираем все гиперссылки
link_elements = soup.select("a[href]")

# Создаем список для записи только необходимых ссылок на тома
toms = []
for link_element in link_elements:
   url = link_element['href']
   # Ищем нужные ссылки на тома по тексту на странице
   if 'Том' in link_element.get_text():
      # Добавляем найденные ссылки в список
      toms.append(url)

# Создаем список для записи только необходимых ссылок на пдф-файлы
pdf_urls = []
# Итерируем через созданный список toms (len(toms) ограничение, сколько томов нам надо)
for i in range(len(toms[:3])):
    # Получаем данные со страницы тома
    tom_page = requests.get(toms[i])
    # Изучаем данные со страницы тома
    tom_soup = BeautifulSoup(tom_page.text, 'html.parser')
    # Выбираем все гиперссылки
    tom_links = tom_soup.select("a[href]")

    # Создаем список для записи только необходимых ссылок на статьи
    articles = []
    for tom_link in tom_links:
        article_url = tom_link['href']
        # Ищем нужные ссылки на статьи по тексту на странице
        if 'PDF' in tom_link.get_text():
          # Добавляем найденные ссылки в список
          articles.append(article_url)

    #Итерируем через созданный список articles (len(articles) ограничение, сколько пдф-файлов нам надо)
    for j in range(len(articles)):
        articles_page = requests.get(articles[j])
        articles_soup = BeautifulSoup(articles_page.text, 'html.parser')
        # Выбираем все гиперссылки
        pdf_links = articles_soup.select("a[href]")

        for pdf_link in pdf_links:
            pdf_file = pdf_link['href']
            # Ищем нужные ссылки на пдф-файлы по тексту на странице
            if 'Скачать этот файл PDF' in pdf_link.get_text():
              # Получаем итоговый список с ссылками на пдф-файлы статьей
              pdf_urls.append(pdf_link.get('href'))

## Формирование корпуса

In [15]:
# Создаем функцию, которая принимает на вход список ссылок и количество слов для извлечения
def make_corpus(pdf_urls, max_words):
  # Инициализируем счетчик слов для дальнейшего сравнения
  total_words = 0
  # Используем функцию enumerate(), которая возвращает объект в формате пар ключ-значение
  for num, url in enumerate(pdf_urls, start=1):
      # Извлекаем данные из определенного ресурса
      response = requests.get(url)
      # Выводим номер скачиваемого файла
      print("Downloading file: ", num)
      # Write content in pdf file
      # Открываем пдф с указанием только для записи (бинарный)
      pdf_file = open("pdf"+str(num)+".pdf", 'wb')
      # Записываем в файл наполнение (метод .content обеспечивает доступ к чистым байтам ответного пейлоада)
      pdf_file.write(response.content)
      # Закрываем файл
      pdf_file.close()
      # Выводим статус скачиваемого файла
      print("File ", num, " downloaded")

      # Обращаемся к необходимому файлу
      pdf_path = f'/content/pdf{num}.pdf'
      # Создаем список для записи текстовых элементов
      text = []
      for pagenum, page in enumerate(extract_pages(pdf_path)):
          # Итеративно обходим элементы, из которых состоит страница
          for element in page:
              # Проверяем, является ли элемент текстовым
              if isinstance(element, LTTextContainer):
                  text.append(text_extraction(element))

      # Итеративно обходим созданный список с текстовыми элементами
      for k in range(len(text)):
        # Задаем условие для поиска аннотаций
        if text[k][0].startswith('Аннотация'):
          # Выбираем элемент по заданному условию
          annotation = text[k][0]
          # Задаем условие на минимальное количество слов в аннотации
          if len(annotation.split()) > 100:
            # Создаем конечный файл "Корпус"
            with open('Корпус.txt', 'a') as file:
              # Записываем в файл аннотацию, убирая симоволы переноса слова и переноса строки
              file.write(annotation.replace('-\n', '').replace('\n', '') + '\n')
              # Добавляем к счетчику количество слов в аннотации
              total_words += len(annotation.split())
              # Сравниваем показатель счетчика с желаемым максмальным количеством слов
              if total_words >= max_words:
                return

In [16]:
# Задаем необходимое количество слов
max_words = 500
# Вызываем функцию и передаем список с ссылками и необходимое количество слов
make_corpus(pdf_urls, max_words)

File  1  downloaded
File  2  downloaded
File  3  downloaded
File  4  downloaded
